## Proceso ETL

In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Auto Sales data.csv", 
    parse_dates=['ORDERDATE'], 
    dayfirst=True)

In [3]:
#Creacion de la nueva columna DAYS_SINCE_LASTORDER basado en la diferencia de tiempo
#Primero elimino la columna existente

data = data.drop('DAYS_SINCE_LASTORDER', axis=1)

#Basado en la diferencia de tiempo creo la nueva columna DAYS_SINCE_LASTORDER

from datetime import datetime
#Especifico la fecha
temp_date = datetime(2020,6,1)
data['DAYS_SINCE_LASTORDER'] = (temp_date - data['ORDERDATE']).dt.days

In [4]:
data.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,CITY,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,DAYS_SINCE_LASTORDER
0,10107,30,95.70,2,2871.00,2018-02-24,Shipped,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,NYC,10022,USA,Yu,Kwai,Small,828
1,10121,34,81.35,5,2765.90,2018-05-07,Shipped,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,Reims,51100,France,Henriot,Paul,Small,756
2,10134,41,94.74,2,3884.34,2018-07-01,Shipped,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,75508,France,Da Cunha,Daniel,Medium,701
3,10145,45,83.26,6,3746.70,2018-08-25,Shipped,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,Pasadena,90003,USA,Young,Julie,Medium,646
4,10168,36,96.66,1,3479.76,2018-10-28,Shipped,Motorcycles,95,S10_1678,Technics Stores Inc.,6505556809,9408 Furth Circle,Burlingame,94217,USA,Hirano,Juri,Medium,582


In [9]:
# Verificar la cantidad de contactos por customers
# Concatenar las columnas de contacto en una nueva columna llamada 'CONTACT_FULLNAME'
data['CONTACT_FULLNAME'] = data['CONTACTFIRSTNAME'] + ' ' + data['CONTACTLASTNAME']

# Mostrar las primeras filas para verificar
print(data[['CONTACTFIRSTNAME', 'CONTACTLASTNAME', 'CONTACT_FULLNAME']].head())


  CONTACTFIRSTNAME CONTACTLASTNAME CONTACT_FULLNAME
0             Kwai              Yu          Kwai Yu
1             Paul         Henriot     Paul Henriot
2           Daniel        Da Cunha  Daniel Da Cunha
3            Julie           Young      Julie Young
4             Juri          Hirano      Juri Hirano


In [20]:
# Borro las columnas sobrantes y voy a mostar los contactos agrupados por clientes
data= data.drop(columns=['CONTACTFIRSTNAME','CONTACTLASTNAME'])

# verificar si hay mas de un contacto por customer
contacto = data.groupby('CUSTOMERNAME')['CONTACT_FULLNAME'].unique().reset_index()

contacto['UNIQUE_CONTACT_COUNT'] = contacto['CONTACT_FULLNAME'].apply(len)

# Filtro customer con mas de un contacto
multiplecontacto = contacto[contacto['UNIQUE_CONTACT_COUNT']>1]

print(multiplecontacto)

Empty DataFrame
Columns: [CUSTOMERNAME, CONTACT_FULLNAME, UNIQUE_CONTACT_COUNT]
Index: []


In [8]:
data['ORDERNUMBER'].unique()

array([10107, 10121, 10134, 10145, 10168, 10180, 10188, 10211, 10223,
       10237, 10251, 10263, 10275, 10285, 10299, 10309, 10318, 10329,
       10341, 10361, 10375, 10388, 10403, 10417, 10103, 10112, 10126,
       10140, 10150, 10163, 10174, 10183, 10194, 10206, 10215, 10228,
       10245, 10258, 10270, 10280, 10291, 10304, 10312, 10322, 10347,
       10357, 10369, 10391, 10411, 10424, 10120, 10210, 10236, 10298,
       10308, 10339, 10374, 10402, 10362, 10105, 10119, 10129, 10143,
       10155, 10167, 10178, 10186, 10197, 10222, 10248, 10261, 10273,
       10283, 10295, 10307, 10316, 10325, 10337, 10350, 10359, 10373,
       10395, 10400, 10414, 10114, 10164, 10175, 10207, 10217, 10229,
       10259, 10281, 10305, 10313, 10334, 10370, 10425, 10108, 10122,
       10135, 10147, 10169, 10181, 10191, 10203, 10225, 10238, 10253,
       10266, 10276, 10287, 10300, 10310, 10320, 10363, 10389, 10419,
       10117, 10127, 10142, 10153, 10165, 10176, 10185, 10196, 10208,
       10220, 10231,

In [22]:
data['CUSTOMERNAME'].unique()

array(['Land of Toys Inc.', 'Reims Collectables', 'Lyon Souveniers',
       'Toys4GrownUps.com', 'Technics Stores Inc.',
       'Daedalus Designs Imports', 'Herkku Gifts', 'Auto Canal Petit',
       'Australian Collectors, Co.', 'Vitachrome Inc.',
       'Tekni Collectables Inc.', 'Gift Depot Inc.', 'La Rochelle Gifts',
       "Marta's Replicas Co.", 'Toys of Finland, Co.',
       'Baane Mini Imports', 'Diecast Classics Inc.',
       'Salzburg Collectables', 'Souveniers And Things Co.',
       'FunGiftIdeas.com', 'UK Collectables, Ltd.',
       'Euro Shopping Channel', 'Volvo Model Replicas, Co',
       'Corrida Auto Replicas, Ltd', 'Dragon Souveniers, Ltd.',
       'Classic Legends Inc.', 'Australian Gift Network, Co',
       'Classic Gift Ideas, Inc', 'Saveley & Henriot, Co.',
       'Canadian Gift Exchange Network', 'West Coast Collectables Co.',
       'Cambridge Collectables Co.', 'Super Scale Inc.',
       'Tokyo Collectables, Ltd', 'Amica Models & Co.',
       'Scandinavian Gift

In [23]:
data['PRODUCTCODE'].unique()

array(['S10_1678', 'S10_1949', 'S10_2016', 'S10_4698', 'S10_4757',
       'S10_4962', 'S12_1099', 'S12_1108', 'S12_1666', 'S12_2823',
       'S12_3148', 'S12_3380', 'S12_3891', 'S12_3990', 'S12_4473',
       'S12_4675', 'S18_1097', 'S18_1129', 'S18_1342', 'S18_1367',
       'S18_1589', 'S18_1662', 'S18_1749', 'S18_1889', 'S18_1984',
       'S18_2238', 'S18_2248', 'S18_2319', 'S18_2325', 'S18_2432',
       'S18_2581', 'S18_2625', 'S18_2795', 'S18_2870', 'S18_2949',
       'S18_2957', 'S18_3029', 'S18_3136', 'S18_3140', 'S18_3232',
       'S18_3259', 'S18_3278', 'S18_3320', 'S18_3482', 'S18_3685',
       'S18_3782', 'S18_3856', 'S18_4027', 'S18_4409', 'S18_4522',
       'S18_4600', 'S18_4668', 'S18_4721', 'S18_4933', 'S24_1046',
       'S24_1444', 'S24_1578', 'S24_1628', 'S24_1785', 'S24_1937',
       'S24_2000', 'S24_2011', 'S24_2022', 'S24_2300', 'S24_2360',
       'S24_2766', 'S24_2840', 'S24_2841', 'S24_2887', 'S24_2972',
       'S24_3151', 'S24_3191', 'S24_3371', 'S24_3420', 'S24_34

# Separacion de archivos

## Tabla pedidos (Orders)

## Tabla detalles orden (Orders_details)

## Tabla productos (Products)

## Tabla Clientes (Clients)

## Tabla Tamaño del trato (deal_size)

In [25]:
#Creamos la tabla customers con la columna incremental de customerID

Customers = data[['CUSTOMERNAME','CONTACT_FULLNAME','PHONE','CITY','ADDRESSLINE1','COUNTRY','POSTALCODE']].drop_duplicates(subset=['CUSTOMERNAME'])

print(Customers)

                           CUSTOMERNAME     CONTACT_FULLNAME  \
0                     Land of Toys Inc.              Kwai Yu   
1                    Reims Collectables         Paul Henriot   
2                       Lyon Souveniers      Daniel Da Cunha   
3                     Toys4GrownUps.com          Julie Young   
4                  Technics Stores Inc.          Juri Hirano   
..                                  ...                  ...   
460        Australian Collectables, Ltd         Sean Connery   
528                    Gift Ideas Corp.            Dan Lewis   
541  Bavarian Collectables Imports, Co.  Michael Donnermeyer   
545                        Royale Belge     Pascale Cartrain   
899             Auto-Moto Classics Inc.        Leslie Taylor   

                 PHONE          CITY                   ADDRESSLINE1  \
0           2125557818           NYC        897 Long Airport Avenue   
1           26.47.1555         Reims             59 rue de l'Abbaye   
2     +33 1 46 62 

In [26]:
#Creamos el ID de customers con un range incremental
Customers['CUSTOMERID'] = range(1, len(Customers) + 1)

Customers.head()

,CUSTOMERNAME,CONTACT_FULLNAME,PHONE,CITY,ADDRESSLINE1,COUNTRY,POSTALCODE,CUSTOMERID
0,Land of Toys Inc.,Kwai Yu,2125557818,NYC,897 Long Airport Avenue,USA,10022,1
1,Reims Collectables,Paul Henriot,26.47.1555,Reims,59 rue de l'Abbaye,France,51100,2
2,Lyon Souveniers,Daniel Da Cunha,+33 1 46 62 7555,Paris,27 rue du Colonel Pierre Avia,France,75508,3
3,Toys4GrownUps.com,Julie Young,6265557265,Pasadena,78934 Hillside Dr.,USA,90003,4
4,Technics Stores Inc.,Juri Hirano,6505556809,Burlingame,9408 Furth Circle,USA,94217,5


In [27]:
#Quitamos los duplicados y creamos la tabla Orders en base a valores unicos
Orders = data[['ORDERNUMBER','ORDERDATE','STATUS','DAYS_SINCE_LASTORDER']].drop_duplicates(subset=['ORDERNUMBER'])

Orders['CUSTOMERID'] = Customers['CUSTOMERID']

Orders.head()

,ORDERNUMBER,ORDERDATE,STATUS,DAYS_SINCE_LASTORDER,CUSTOMERID
0,10107,2018-02-24,Shipped,828,1.0
1,10121,2018-05-07,Shipped,756,2.0
2,10134,2018-07-01,Shipped,701,3.0
3,10145,2018-08-25,Shipped,646,4.0
4,10168,2018-10-28,Shipped,582,5.0


In [28]:
#CREO la pequeña tabla de Dealsize

DealSize = data[['DEALSIZE']].drop_duplicates(subset=['DEALSIZE'])
DealSize['DEALSIZEID'] = range(1, len(DealSize)+1)
print(DealSize)

   DEALSIZE
0     Small
2    Medium
18    Large


In [29]:
# Creo el id dealsize
DealSize['DEALSIZEID'] = range(1, len(DealSize)+1)
print(DealSize)

   DEALSIZE  DEALSIZEID
0     Small           1
2    Medium           2
18    Large           3


In [32]:
# Voy a crear la tabla OrderDetails

OrdersDetails = data[['ORDERNUMBER','QUANTITYORDERED','PRODUCTCODE','PRICEEACH','SALES']]

In [33]:
#CREO EL DETAILSID Y COLOCO LA COLUMNA DEALSIZEID

OrdersDetails['DETAILSID'] = range(1, len(OrdersDetails)+1)

OrdersDetails.insert(0, 'DETAILSID', OrdersDetails.pop('DETAILSID'))

OrdersDetails['DEALSIZEID'] = DealSize['DEALSIZEID']

OrdersDetails.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_14528\2155205232.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  OrdersDetails['DETAILSID'] = range(1, len(OrdersDetails)+1)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14528\2155205232.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  OrdersDetails['DEALSIZEID'] = DealSize['DEALSIZEID']


,DETAILSID,ORDERNUMBER,QUANTITYORDERED,PRODUCTCODE,PRICEEACH,SALES,DEALSIZEID
0,1,10107,30,S10_1678,95.70,2871.00,1.0
1,2,10121,34,S10_1678,81.35,2765.90,NaN
2,3,10134,41,S10_1678,94.74,3884.34,2.0
3,4,10145,45,S10_1678,83.26,3746.70,NaN
4,5,10168,36,S10_1678,96.66,3479.76,NaN


In [34]:
#Creo la tabla Products quitando los duplicados

Products = data[['PRODUCTCODE','PRODUCTLINE','MSRP']].drop_duplicates(subset=['PRODUCTCODE'])
Products.head()

,PRODUCTCODE,PRODUCTLINE,MSRP
0,S10_1678,Motorcycles,95
24,S10_1949,Classic Cars,214
50,S10_2016,Motorcycles,118
74,S10_4698,Motorcycles,193
98,S10_4757,Classic Cars,136


In [36]:
#Guardamos los dataframe en su respectivo arhivo csv para la importar a SQL 

Orders.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Orders.csv", index=False)

OrdersDetails.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Orders_Details.csv", index=False)

Products.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Products.csv", index=False)

Customers.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Customers.csv", index=False)

DealSize.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Deal_Size.csv", index=False)